## Gabor filter feature extractor simulations.

 - The configs for these simulations are available under projects/initial_demo/experiments/base.py.

 - Note that in these configs, the sensor module receives a whole RGBD image as input and segments it down to a patch which is sent to the learning module (and thereby feature extractor). This is a hack for now while the BugEye sensor is being implemented.

 - Each action has an amount associated with it (e.g. turn left 30 degrees, move forward .5 meters, etc). This experiment is assuming random sticky actions with the actuation amount changing each step. The actuation amount sampler can be changed in base.py.

In [ ]:
%load_ext autoreload

%autoreload 2

import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
import torch
# from torch_geometric.data import Data

import habitat_sim
from habitat_sim import AgentConfiguration, CameraSensorSpec

from tbp.monty.simulators.habitat import HabitatSim
from tbp.monty.frameworks.environments.habitat import SingleSensorAgent

from tbp.monty.frameworks.experiments.monty_experiment import MontyExperiment

from experiments import CONFIGS

from dev_utils import (
    habitat_utils as hu,
    model_utils as mu,
    plot_utils as pu
)

### Set up an experiment with gabor filter feature extractor

In [ ]:
# setup_experiment removes keys from the config as it processes some of them, so use a copy
gabor_config = copy.deepcopy(CONFIGS["base_gf"])

In [ ]:
# setup_experiment removes keys from the config as it processes some of them, so use a copy
gabor_config = copy.deepcopy(CONFIGS["base_gf"])
my_exp = MontyExperiment()
my_exp.setup_experiment(gabor_config)

### Verify the train_epoch method works

Note that for these simply early sims, I am printing out each action. You can turn this off by setting verbose=False in models/motor_policies/PredefinedPolicy.

In [ ]:
my_exp.train_epoch()

### Now create a new experiment, and instead of doing train_epoch, use the developer util get_n_model_outputs. This allows you to save the observations returned by the loader, and the feature extractor results easily.

Note, you do need to create a new experiment, otherwise you will hit an error where the dataloader says you passed max_steps, or PredefinedPolicy index out of range, since you did in fact just iterate over the whole list of actions.

In [ ]:
# setup_experiment removes keys from the config as it processes some of them, so use a copy
gabor_config = copy.deepcopy(CONFIGS["base_gf"])
my_exp = MontyExperiment()
my_exp.setup_experiment(gabor_config)

In [ ]:
obs, outs = mu.get_n_model_outputs(my_exp.dataloader, my_exp.model, 15)

### Visualize results. Plot all 15 observations (rgba channels) and denote the patch of input the feature extractor receives with a blue square. 

In [ ]:
box_params = CONFIGS["base_gf"]["sensor_module_configs"]["sensor_module_1"]["preprocessor_args"]

xy = (box_params["col_start"], box_params["row_start"])
width = box_params["col_end"] - box_params["col_start"]
height = box_params["row_end"] - box_params["row_start"]

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(16,8))

c = 0
for i in range(3):
    for j in range(5):
        ax[i, j].imshow(obs[c]["agent_id_0"]["sensor_id_0"]["rgba"])
        rect = patches.Rectangle(xy, width, height, linewidth=1, edgecolor="w")
        ax[i, j].add_patch(rect)
        c += 1

## Now look at the feature maps returned by gabor filters.

You can choose to store feature maps of observations by setting store_convs to True in GaborFilterFeatureExtractor. You would do this by setting it as an argument in FeatureExtractorArgs in the config.

First, look at the feature map on the first (0th) observation, just one channel, all filter orientations.

In [ ]:
# 0th time step, 0th sensor module output, 0th (r) channel, all filters
pu.visualize_filters(my_exp.model.learning_modules[0].feature_extractor.convs[0][0][0])

Now look at ALL observations, but only a single filter. Also, check out the depth channel instead of RGBA.

In [ ]:
# every time step, 0th sensor module output, -1=depth channel, 8th filter
fms = [my_exp.model.learning_modules[0].feature_extractor.convs[i][0][-1][8] for i in range(16)]
pu.visualize_filters(fms)